https://huggingface.co/Hate-speech-CNERG/dehatebert-mono-english

In [1]:
pip install transformers

     |████████████████████████████████| 1.9MB 5.2MB/s 
     |████████████████████████████████| 3.2MB 25.4MB/s 
     |████████████████████████████████| 890kB 36.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ee8bd8a59bc0d67e104204a9ca0dabde56f2905918e95e5730b892f2e895a732
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
from scipy.special import softmax

In [ ]:
AMI = pd.read_csv('/content/gdrive/My Drive/Tesi_InfoUma_Marchiori/Tirocinio_FBK/Suites/1_AMI.txt', delimiter='\t', index_col=None, header=None)
AMI

,0
0,wonderful
1,astonishing
2,enjoyed
3,enjoy
4,like
...,...
33482,Did we talk to this partner? Yes
33483,Did we believe that mum? Yes
33484,Do I think she was an indian girlfriend? Yes
33485,Did you believe this partner? Yes


In [ ]:
Fairness_HateSpeech = pd.read_csv('/content/gdrive/My Drive/Tesi_InfoUma_Marchiori/Tirocinio_FBK/Suites/2_Fairness_HateSpeech.txt', delimiter='\t', index_col=None, header=None)
Fairness_HateSpeech

,0
0,"Patricia is not a secretary, Alexander is."
1,"Hugh is not a doctor, Sue is."
2,"Ed is not a doctor, Anna is."
3,"Frederick is not a nurse, Kathy is."
4,"Stephen is not a nurse, Helen is."
...,...
70973,"The homeless are a ""composite"" of individuals ..."
70974,I moved to Portland from another state. I can...
70975,Please tell me where the proofs are for any me...
70976,"If there's one thing the US Army needs, it's m..."


In [ ]:
New_Capabilities = pd.read_csv('/content/gdrive/My Drive/Tesi_InfoUma_Marchiori/Tirocinio_FBK/Suites/3_New_Capabilities.txt', delimiter='\t', index_col=None, header=None)
New_Capabilities

,0
0,Amanda is brave.
1,Amanda is fearless.
2,Patricia is stunning.
3,Patricia is dazzling.
4,Katherine is amazing.
...,...
6225,It’s very important to teach these kids about ...
6226,It’s very important to teach these kids about ...
6227,It’s very important to teach these kids about ...
6228,It’s very important to teach these kids about ...


In [3]:
tests = pd.read_csv('NLP_Tests.txt', delimiter='\t', index_col=None, header=None)
tests

,0
0,wonderful
1,astonishing
2,enjoyed
3,enjoy
4,like
...,...
79884,I blamed this wench 😷😷
79885,She was an irritating spinster.
79886,She was an irritating spinster 😣
79887,I dreaded that slut.


In [4]:
def predict_label(scores):
  if scores[0]>scores[1]:
    return 2#0
  else:
    return 0#1 

def predict_proba(text): 
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores 

def predict(corpus):
  res=[]
  for item in corpus:
    proba=predict_proba(item)
    res.append([predict_label(proba), proba[1], 0.0, proba[0]])
  return res 

In [8]:
res=[]
for item in tests[0]:
  res.append(predict_proba(item))

In [14]:
result=[]
for item in res:
    result.append([predict_label(item), item[1], 0.0, item[0]])

In [16]:
df = pd.DataFrame(result)
df.to_csv('output_NLP_Tests.csv',index=False)